In [ ]:
import webbrowser as wb
import speech_recognition as sr
from time import ctime
import os
from gtts import gTTS
from googlesearch import search
import pygame
import uuid

def speak(audioString):
    print(audioString)
    tts = gTTS(text=audioString, lang='en')
    filename = f"audio_{uuid.uuid4()}.mp3"  # unique filename every time
    tts.save(filename)

    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

    clock = pygame.time.Clock()
    while pygame.mixer.music.get_busy():
        clock.tick(10)

    pygame.mixer.music.stop()
    pygame.quit()

    try:
        os.remove(filename)
    except PermissionError:
        print(f"Warning: Could not delete {filename} immediately.")

    speak("Hold on" + name + ", I will redirect you to Facebook.")
    wb.open("https://www.facebook.com")

def youtube(name):
    speak("Hold on " + name + ", I'm opening YouTube for you.")
    wb.open("https://www.youtube.com")

def gmail(name):
    speak("Just a minute " + name + ", bringing you to Gmail.")
    wb.open("https://www.gmail.com")

def locations(data, name):
    words = data.split(" ")
    if len(words) >= 3:
        location = " ".join(words[2:])
        speak("Hold on " + name + ", I will show you where " + location + " is.")
        wb.open("https://www.google.com/maps/place/" + location)
    else:
        speak("Sorry " + name + ", I didn't catch the location.")

def search_data(data, name):
    speak("Wait a moment " + name + ", I'm searching for you.")
    wb.open("https://www.google.com/search?q=" + data)

def recordAudio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Adjusting for ambient noise...")
        r.adjust_for_ambient_noise(source, duration=1)
        print("Listening...")
        audio = r.listen(source)

    data = ""
    try:
        data = r.recognize_google(audio)
        print("You said: " + data)
        speak("You said: " + data)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        speak("Sorry, I couldn't understand what you said.")
    except sr.RequestError as e:
        print("Could not request results from Google; {0}".format(e))
        speak("I'm having trouble connecting to Google.")

    return data.lower()

def PA(data, name):
    if "what is your name" in data:
        speak("I'm Clara.")

    elif "how are you" in data:
        speak("I am fine, and you?")

    elif "what time is it" in data:
        speak(ctime())

    elif "where is" in data:
        locations(data, name)

    elif "facebook" in data:
        facebook(name)

    elif "music" in data or "youtube" in data:
        youtube(name)

    elif "mail" in data or "gmail" in data:
        gmail(name)

    elif "search for" in data:
        search_data(data, name)

    elif "goodbye" in data or "exit" in data:
        speak("Goodbye, " + name + "! Take care.")
        exit()

# --- Program start ---
speak("What's your name?")
name = input("Your name: ")
speak("Hi " + name + "! What can I do for you?")

while True:
    print("Speak...")
    data = recordAudio()
    print("Processing...")

    if data == "":
        continue

    if "exit" in data or "goodbye" in data:
        speak("Goodbye, " + name + "! Take care.")
        break
    else:
        PA(data, name)

print("Program ended.")